# ML Surrogates for Chemical Processes with OMLT
This notebook illustrates the use of TensorFlow Keras and OMLT to produce an ML surrogate based on data from a chemical process flowsheet.

There are several reasons to build surrogate models for complex processes, even when higher fidelity models already exist (e.g., reduce model size, improve convergence reliability, replace models with externally compiled code and make them fully-equation oriented).

In this example, we have an existing model for an auto-thermal reformer flowsheet that has been built using the IDAES-PSE package. IDAES-PSE is a Python package that is built on Pyomo and provides a framework for equation-oriented modeling and analysis of advanced energy systems. We use this package to generate data for our systems, and then we utilize this data in an optimization problem with OMLT. To learn more about IDAES, see the [IDAES-PSE Github Page](https://github.com/IDAES/IDAES-PSE) or [IDAES Read-the-docs](https://idaes-pse.readthedocs.io/en/stable/).

## The Auto-thermal Reformer Process

The figure below shows the reformer process as modeled in IDAES.

![Reformer Flowsheet](../images/reformer.png)

This model has 12 outputs of interest, the steam flowrate, the reformer duty, and the properties of the outlet stream, including temperature, pressure, and composition. We are interested modeling how these outputs change as a function of two operating (or input) variables: 
- the fraction of natural gas that bypasses the reformer
- steam to natural gas flow ratio

We have already used IDAES to generate a CSV file that contains the input and output data for 2800 data points for our system.

In this example, we will train a neural network model with sigmoid activations from our process data and then demonstrate that we can solve an optimization problem with that surrogate model. In realistic applications, this surrogate model would form part of a design or operations problem with a much larger flowsheet. 

## Library Setup
This notebook assumes you have a working Tensorflow environment in addition to necessary Python packages described here. We use Keras to train neural networks of interest for our example which requires the Python Tensorflow package. The neural networks are then formulated in Pyomo using OMLT which therefore requires working Pyomo and OMLT installations.

The required Python libraries used this notebook are as follows: <br>
- `pandas`: used for data import and management <br>
- `tensorflow`: the machine learning language we use to train our neural network
- `pyomo`: the algebraic modeling language for Python, it is used to define the optimization model passed to the solver
- `onnx`: used to express trained neural network models
- `omlt`: The package this notebook demonstates. OMLT can formulate machine learning models (such as neural networks) within Pyomo

**NOTE:** This notebook also assumes you have a working MIP solver executable (e.g., CBC, Gurobi) to solve optimization problems in Pyomo. The open-source solver IPOPT is called by default.

In [22]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # suppress CUDA warnings from tensorflow

# import the necessary packages
import pandas as pd
import pyomo.environ as pyo
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from omlt import OffsetScaling, OmltBlock
from omlt.io.keras import load_keras_sequential
from omlt.neuralnet import FullSpaceSmoothNNFormulation

In [23]:
# read in our csv data
columns = [
    "Bypass Fraction",
    "NG Steam Ratio",
    "Steam Flow",
    "Reformer Duty",
    "AR",
    "C2H6",
    "C3H8",
    "C4H10",
    "CH4",
    "CO",
    "CO2",
    "H2",
    "H2O",
    "N2",
]
df = pd.read_csv("../data/reformer.csv", usecols=columns)
print(df)

      Bypass Fraction  NG Steam Ratio  Steam Flow  Reformer Duty        AR  \
0                 0.8        0.800000    0.193898    9806.732716  0.002662   
1                 0.8        0.810526    0.196449    9846.047501  0.002660   
2                 0.8        0.821053    0.199000    9885.419259  0.002657   
3                 0.8        0.831579    0.201552    9924.849127  0.002654   
4                 0.8        0.842105    0.204103    9964.338177  0.002651   
...               ...             ...         ...            ...       ...   
2795              0.1        1.157895    1.262887   39771.876388  0.004086   
2796              0.1        1.168421    1.274368   39989.582852  0.004080   
2797              0.1        1.178947    1.285849   40207.531167  0.004073   
2798              0.1        1.189474    1.297330   40425.721366  0.004067   
2799              0.1        1.200000    1.308811   40644.153425  0.004060   

          C2H6      C3H8     C4H10       CH4        CO       CO

In [24]:
# separate the data into inputs and outputs
inputs = ["Bypass Fraction", "NG Steam Ratio"]
outputs = [
    "Steam Flow",
    "Reformer Duty",
    "AR",
    "C2H6",
    "C3H8",
    "C4H10",
    "CH4",
    "CO",
    "CO2",
    "H2",
    "H2O",
    "N2",
]
dfin = df[inputs]
dfout = df[outputs]

In [25]:
# We scale the data for improved training, however, we want to formulate
# our optimizaton problem on the original variables. Therefore, we keep
# the scaling parameters to use later in our optimization formulation

x_offset, x_factor = dfin.mean().to_dict(), dfin.std().to_dict()
y_offset, y_factor = dfout.mean().to_dict(), dfout.std().to_dict()

dfin = (dfin - dfin.mean()).divide(dfin.std())
dfout = (dfout - dfout.mean()).divide(dfout.std())

# capture the minimum and maximum values of the scaled inputs
# so we don't use the model outside the valid range
scaled_lb = dfin.min()[inputs].to_numpy()
scaled_ub = dfin.max()[inputs].to_numpy()

In [26]:
# create our Keras Sequential model
nn = Sequential(name="reformer_sigmoid_4_20")
nn.add(Dense(units=20, input_dim=len(inputs), activation="sigmoid"))
nn.add(Dense(units=20, activation="sigmoid"))
nn.add(Dense(units=20, activation="sigmoid"))
nn.add(Dense(units=20, activation="sigmoid"))
nn.add(Dense(units=len(outputs)))
nn.compile(optimizer=Adam(), loss="mse")

/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
# train our model
x = dfin.to_numpy()
y = dfout.to_numpy()

history = nn.fit(x, y, epochs=100)

Epoch 1/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 988us/step - loss: 1.1144
Epoch 2/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - loss: 0.9900
Epoch 3/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9766
Epoch 4/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8390  
Epoch 5/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - loss: 0.2823
Epoch 6/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1576  
Epoch 7/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 891us/step - loss: 0.1403
Epoch 8/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 933us/step - loss: 0.1267
Epoch 9/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - loss: 0.1145
Epoch 10/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - loss: 0.1074
Epoch 11/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1041
Epoch 12/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step - loss: 0.1006
Epoch 13/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 982us/step - loss: 0.0973
Epoch 14/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - loss: 0.0939
Epoch 15/100
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 870u

In [28]:
# save the model to disk
# While not technically necessary, this shows how we can load a previously saved model into
# our optimization formulation)
nn.save("reformer_nn.keras")

## Optimization Problem
In this small example, we will formulate a simple optimization problem that seeks to maximize the concentration of Hydrogen in the outlet while placing an upper bound on the Nitrogen concentration.

In [29]:
# first, create the Pyomo model
m = pyo.ConcreteModel()

In [30]:
# create the OmltBlock to hold the neural network model
m.reformer = OmltBlock()

In [31]:
# load the Keras model
nn_reformer = keras.models.load_model("reformer_nn.keras", compile=False)

# Note: The neural network is in the scaled space. We want access to the
# variables in the unscaled space. Therefore, we need to tell OMLT about the
# scaling factors
scaler = OffsetScaling(
    offset_inputs={i: x_offset[inputs[i]] for i in range(len(inputs))},
    factor_inputs={i: x_factor[inputs[i]] for i in range(len(inputs))},
    offset_outputs={i: y_offset[outputs[i]] for i in range(len(outputs))},
    factor_outputs={i: y_factor[outputs[i]] for i in range(len(outputs))},
)

scaled_input_bounds = {i: (scaled_lb[i], scaled_ub[i]) for i in range(len(inputs))}

# create a network definition from the Keras model
net = load_keras_sequential(
    nn_reformer, scaling_object=scaler, scaled_input_bounds=scaled_input_bounds
)

# create the variables and constraints for the neural network in Pyomo
m.reformer.build_formulation(FullSpaceSmoothNNFormulation(net))

In [32]:
# now add the objective and the constraints
h2_idx = outputs.index("H2")
n2_idx = outputs.index("N2")
m.obj = pyo.Objective(expr=m.reformer.outputs[h2_idx], sense=pyo.maximize)
m.con = pyo.Constraint(expr=m.reformer.outputs[n2_idx] <= 0.34)

In [33]:
# now solve the optimization problem
solver = pyo.SolverFactory("ipopt")
status = solver.solve(m, tee=True)

Ipopt 3.14.16: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.1.

Number of nonzeros in equality constraint Jacobian...:     1812
Number of nonzeros in inequality constraint Jacobian.:        1
Number of nonzeros in Lagrangian Hessian.............:       80

Total number of variables............................:      214
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      190
                     variables with only upper bounds:        0
Total number of equality constraints.................:      212
Total number

In [34]:
print("Bypass Fraction:", pyo.value(m.reformer.inputs[0]))
print("NG Steam Ratio:", pyo.value(m.reformer.inputs[1]))
print("H2 Concentration:", pyo.value(m.reformer.outputs[h2_idx]))
print("N2 Concentration:", pyo.value(m.reformer.outputs[n2_idx]))

Bypass Fraction: 0.1000002111229052
NG Steam Ratio: 1.1131313973800456
H2 Concentration: 0.33184454733260904
N2 Concentration: 0.3400000044198399
